# Anomaly Detection: Visual Quality Inspection in the Industrial Domain

Manual anomaly detection is time and labor-intensive, which limits its applicability on large volumes of data that are typical in industrial settings. Application of artificial intelligence and machine learning is transforming Industrial Internet of Things (IIoT) segments by enabling higher productivity, better insights, less downtime, and superior product quality.

The goal of this anomaly detection reference use case is to provide AI-powered visual quality inspection on high resolution input images by identifying rare, abnormal events such as defects in a part being manufactured on an industrial production line. Use this reference solution as-is on your dataset, curate it to your needs by fine-tuning the models, change configurations to get improved performance, and modify it to meet your productivity and performance goals by making use of the modular architecture and realize superior performance using the Intel optimized software packages and libraries for Intel hardware that are built into the solution.
 

## **Table of Contents**
- [Technical Overview](#technical-overview)
    - [DataSet](#DataSet)
- [Validated Hardware Details](#validated-hardware-details)
- [Software Requirements](#software-requirements)
- [How it Works?](#how-it-works)
- [Get Started](#get-started)
    - [Download the Workflow Repository](#Download-the-Workflow-Repository)
    - [Download the Transfer Learning Tool](#Download-the-Transfer-Learning-Tool)
- [Ways to run this reference use case](#Ways-to-run-this-reference-use-case)
    - [Run Using Bare Metal](#run-using-bare-metal) 
- [Expected Output](#expected-output)
- [Summary and Next Steps](#summary-and-next-steps)
    - [Adopt to your dataset](#adopt-to-your-dataset)
    - [Adopt to your model](#adopt-to-your-model)
- [Learn More](#learn-more)
- [Support](#support)

<a id="technical-overview"></a> 
## Technical Overview
Classic and modern anomaly detection techniques have certain challenges:

- Feature engineering needs to be performed to extract representations from the raw data. Traditional ML techniques rely on hand-crafted features that may not always generalize well to other settings.
- Classification techniques require labeled training data, which is challenging because anomalies are typically rare occurrences and obtaining it increases the data collection & annotation effort.
- Nature of anomalies can be arbitrary and unknown where failures or defects occur for a variety of unpredictable reasons, hence it may not be possible to predict the type of anomaly.

To overcome these challenges and achieve state-of-the-art performance, we present an unsupervised, mixed method end-to-end fine-tuning & inference reference solution for anomaly detection where a model of normality is learned from defect-free data in an unsupervised manner, and deviations from the models are flagged as anomalies. This reference use case is accelerated by Intel optimized software and is built upon easy-to-use Intel Transfer Learning Tool APIs.

<a id="DataSet"></a> 
### DataSet
[MVTec AD](https://www.mvtec.com/company/research/datasets/mvtec-ad) is a dataset for benchmarking anomaly detection methods focused on visual quality inspection in the industrial domain. It contains over 5000 high-resolution images divided into ten unique objects and five unique texture categories. Each category comprises a set of defect-free training images and a test set of images with various kinds of defects as well as defect-free images. There are 73 different types of anomalies in the form of defects or structural deviations present in these objects and textures.

More information can be in the paper [MVTec AD – A Comprehensive Real-World Dataset for Unsupervised Anomaly Detection](https://www.mvtec.com/fileadmin/Redaktion/mvtec.com/company/research/datasets/mvtec_ad.pdf)

![Statistical_overview_of_the_MVTec_AD_dataset](./assets/mvtec_dataset_characteristics.JPG)
<br>
*Table 1:  Statistical overview of the MVTec AD dataset. For each category, the number of training and test images is given together with additional information about the defects present in the respective test images. [Source](https://www.mvtec.com/fileadmin/Redaktion/mvtec.com/company/research/datasets/mvtec_ad.pdf)*

<a id="validated-hardware-details"></a> 
## Validated Hardware Details
There are workflow-specific hardware and software setup requirements depending on how the workflow is run. Bare metal development system and Docker image running locally have the same system requirements. 

| Recommended Hardware         | Precision  |
| ---------------------------- | ---------- |
| Intel® 4th Gen Xeon® Scalable Performance processors| float32, bfloat16 |
| Intel® 1st, 2nd, 3rd Gen Xeon® Scalable Performance processors| float32 |

<a id="software-requirements"></a> 
## Software Requirements 
Linux OS (Ubuntu 20.04) is used in this reference solution. Make sure the following dependencies are installed.

1. `sudo apt update`
1. `sudo apt-get install -y libgl1 libglib2.0-0`
1. pip/conda OR python3.9-venv
1. git



<a id="how-it-works"></a> 
## How It Works?

This reference use case uses a deep learning based approach, named deep-feature modeling (DFM) and falls within the broader area of out-of-distribution (OOD) detection i.e. when a model sees an input that differs from its training data, it is marked as an anomaly. Learn more about the approach [here.](https://arxiv.org/pdf/1909.11786.pdf) 

The use case provides 3 options for modeling of the vision subtask:
* **Pre-trained backbone:** uses a deep network (ResNet-50v1.5 in this case) that has been pretrained on large visual datasets such as ImageNet
* **SimSiam self-supervised learning:** is a contrastive learning method based on Siamese networks. It learns meaningful representation of dataset without using any labels. SimSiam requires a dataloader such that it can produce two different augmented images from one underlying image. The end goal is to train the network to produce same features for both images. It takes a ResNet model as the backbone and fine-tunes the model on the augmented dataset to get closer feature embeddings for the use case. Read more [here.](https://arxiv.org/pdf/2011.10566.pdf)
* **Cut-Paste self-supervised learning:** is a contrastive learning method similar to SimSiam but differs in the augmentations used during training. It take a ResNet model as backbone and fine-tunes the model after applying a data augmentation strategy that cuts an image patch and pastes at a random location of a large image. This allows us to construct a high performance model for defect detection without presence of anomalous data. Read more [here.](https://arxiv.org/pdf/2104.04015.pdf)

![visual_quality_inspection_pipeline](./assets/visual_quality_inspection_pipeline.JPG)
*Figure 1: Visual quality inspection pipeline. Above diagram is an example when using SimSiam self-supervised training.*

Training stage only uses defect-free data. Images are loaded using a dataloader and shuffling, resizing & normalization processing is applied. Then one of the above stated transfer learning technique is used to fine-tune a model and extract discriminative features from an intermediate layer. A PCA kernel is trained over these features to reduce the dimension of the feature space while retaining 99% variance. This pre-processing of the intermediate features of a DNN is needed to prevent matrix singularities and rank deficiencies from arising.

During inference, the feature from a test image is generated through the same network as before. We then run a PCA transform using the trained PCA kernel and apply inverse transform to recreate original features and generate a feature-reconstruction error score, which is the norm of the difference between the original feature vector and the pre-image of its corresponding reduced embedding. Any image with an anomaly will have a high error in reconstructing original features due to features being out of distribution from the defect-free training set and will be marked as anomaly. The effectiveness of these scores in distinguishing the good images from the anomalous images is assessed by plotting the ROC curve, which is a plot of the true positive rate (TPR) of the classifier against the false positive rate (FPR) as the classification score-threshold is varied. The AUROC metric summarizes this curve between 0 to 1, with 1 indicating perfect classification.



**Architecture:**
![Visual_quality_inspection_layered_architecture](./assets/Visual_quality_inspection_layered_architecture.JPG)

### Highlights of Visual Quality Inspection Reference Use Case
- The use case is presented in a modular architecture. To improve productivity and reduce time-to-solution, transfer learning methods are made available through an independent workflow that seamlessly uses Intel Transfer Learning Tool APIs underneath and a config file allows the user to change parameters and settings without having to deep-dive and modify the code.
- There is flexibility to select any pre-trained model and any intermediate layer for feature extraction.
- The use case is enabled with Intel optimized foundational tools.


<a id="get-started"></a> 
## Get Started

<a id="Download-the-Workflow-Repository"></a> 
### Download the Workflow Repository

Create a working directory for the reference use case and clone the [Visual Quality Inspection Workflow](https://github.com/intel/visual-quality-inspection) repository into your working directory.
```
git clone https://github.com/intel/visual-quality-inspection
cd visual-quality-inspection
```

<a id="Download-the-Transfer-Learning-Tool"></a> 
### Download the Transfer Learning Tool
```
git submodule update --init --recursive
export PYTHONPATH=transfer-learning/
```

In [ ]:
!git submodule update --init --recursive

In [ ]:
%env PYTHONPATH=transfer-learning/
import os
print(os.environ["PYTHONPATH"])

<a id="Ways-to-run-this-reference-use-case"></a> 
# Ways to run this reference use case

This reference kit offers three options for running the fine-tuning and inference processes:

- Docker
- Argo Workflows on K8s Using Helm
- [Bare Metal](#run-using-bare-metal)

Details about Bare Metal method can be found below.

<a id="run-using-bare-metal"></a> 
# Run Using Bare Metal

### 1. Create environment and install software packages

**Opton 1: Using conda:**

In [ ]:
conda create -n anomaly_det_refkit python=3.9 ipykernel -y -q

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

NOTE: Restart kernel and wait some seconds, and change kernel to conda env:anomaly_det_refkit

In [ ]:
# Verify anomaly_det_refkit environment is active
!conda env list

In [ ]:
!pip install -r requirements.txt

**Option 2: Using virtualenv:**

```
python3 -m venv anomaly_det_refkit
source anomaly_det_refkit/bin/activate
pip install -r requirements.txt
```

In [ ]:
!python3 -m venv anomaly_det_refkit  
!anomaly_det_refkit/bin/python3 -m pip install ipykernel
!anomaly_det_refkit/bin/python3 -m ipykernel install --user --name anomaly_det_refkit

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

**NOTE:** Restart kernel and wait some seconds, and change kernel to anomaly_det_refkit. 

In [ ]:
!anomaly_det_refkit/bin/pip install -r requirements.txt

### 2. Download the dataset

Download the mvtec dataset using Intel Model Zoo dataset download API

In [ ]:
!git clone https://github.com/IntelAI/models.git

Install dependencies and download the dataset

In [ ]:
%cd models/datasets/dataset_api/
!pip install -r requirements.txt

**Note**: Run on terminal with anomaly_det_refkit virtual environment activated
to review terms and conditions
```
cd models/datasets/dataset_api/
./setup.sh
```


In [ ]:
!python dataset.py -n mvtec-ad --download -d ../../../

Extract the tar file

In [ ]:
%cd ../../../
!mkdir mvtec_dataset
!tar -xf mvtec_anomaly_detection.tar.xz --directory mvtec_dataset

### 3. Select parameters and configurations

Select the parameters and configurations in the [finetuning.yaml](configs/README.md) file.

NOTE: 
When using SimSiam self supervised training, download the Sim-Siam weights based on ResNet50 model and place under simsiam directory:

In [ ]:
!mkdir simsiam
!wget --directory-prefix=/simsiam/ https://dl.fbaipublicfiles.com/simsiam/models/100ep-256bs/pretrain/checkpoint_0099.pth.tar -o ./simsiam/checkpoint_0099.pth.tar

### 4. Running the end-to-end use case 

Using Transfer Learning Tool based fine-tuning:

In finetuning.yaml, set **'fine_tune'** flag to true. If you downloaded the data from [DataSet](#DataSet) **change ./data/ to ./mvtec_dataset/** and set the pretrained/simsiam/cutpaste settings accordingly.
Change other settings as intended in finetuning.yaml to run different configurations.

In [ ]:
%env PYTHONPATH=transfer-learning/
!python anomaly_detection.py --config_file ./configs/finetuning.yaml

<a id="expected-output"></a> 
## Expected Output

```
+------------+------------------------+-------+--------------+
|  Category  | Test set (Image count) | AUROC | Accuracy (%) |
+------------+------------------------+-------+--------------+
|   BOTTLE   |           83           | 99.92 |     98.8     |
|   CABLE    |          150           | 94.36 |    88.67     |
|  CAPSULE   |          132           | 95.33 |    87.12     |
|   CARPET   |          117           | 91.65 |    83.76     |
|    GRID    |           78           |  86.3 |    82.05     |
|  HAZELNUT  |          110           | 99.25 |    97.27     |
|  LEATHER   |          124           |  99.9 |    98.39     |
| METAL_NUT  |          115           |  93.3 |    90.43     |
|    PILL    |          167           | 96.02 |    86.83     |
|   SCREW    |          160           |  83.3 |    81.88     |
|    TILE    |          117           | 98.81 |    99.15     |
| TOOTHBRUSH |           42           | 96.11 |     88.1     |
| TRANSISTOR |          100           | 96.42 |     91.0     |
|    WOOD    |           79           |  99.3 |    97.47     |
|   ZIPPER   |          151           | 97.16 |    90.07     |
+------------+------------------------+-------+--------------+
```
*Above results are on single node Dual socket 4th Generation Intel Xeon Scalable 8480+ (codenamed: Sapphire Rapids) Processor. 56 cores per socket, Intel® Turbo Boost Technology enabled, Intel® Hyper-Threading Technology enabled, 1024 GB memory (16x64GB), Configured Memory speed=4800 MT/s, INTEL SSDSC2BA012T4, CentOS Linux 8, BIOS=EGSDCRB.86B.WD.64.2022.29.7.13.1329, CPU Governor=performance, intel-extension-for-pytorch v2.0.0, torch 2.0.0, scikit-learn-intelex v2023.1.1, pandas 2.0.1. Configuration: precision=bfloat16, batch size=32, features extracted from pretrained resnet50v1.50 model.*


<a id="summary-and-next-steps"></a> 
## Summary and Next Steps


The reference use case above demonstrates an Anomaly Detection approach using deep feature extraction and out-of-distrabution detection. It uses a tunable, modular workflow for fine-tuning the model & extractingits features, both of which uses the Intel® Transfer Learning Tool underneath. For optimal performance on Intel architecture, the scripts are also enabled with Intel extension for PyTorch, Intel extension for scikit-learn and has an option to run bfloat16 on 4th Gen Intel Xeon scalable processors using Intel® Advanced Matrix Extensions (Intel® AMX).

### How to customize this use case
Tunable configurations and parameters are exposed using yaml config files allowing users to change model training hyperparameters, datatypes, paths, and dataset settings without having to modify or search through the code.

<a id="adopt-to-your-dataset"></a> 
#### Adopt to your dataset

This reference use case can be easily deployed on a different or customized dataset by simply arranging the images for training and testing in the following folder structure (Note that this approach only uses good images for training):

In [11]:
## Code to create diagram.
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt

def mm(graph):
  graphbytes = graph.encode("ascii")
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  display(Image(url="https://mermaid.ink/img/" + base64_string))

mm("""
graph TD;
    dataset-->train;
    dataset-->test;
    train-->Good;
    test-->crack;
    test-->good;
    test-->joint;
    test-->dot;
    test-->other_anomalies;
""")

For example, to run it for a [Marble Surface Anomaly Detection dataset](https://www.kaggle.com/datasets/wardaddy24/marble-surface-anomaly-detection-2) in Kaggle, download the dataset and update the train folder to only include the 'good' folder. Move the sub-folders with anomaly images in train folder to either the corresponding test folders or delete them.

<a id="adopt-to-your-model"></a> 
#### Adopt to your model

#### 1. Change to a different pre-trained model from Torchvision:
Change the 'model/name' variable in configs/finetuning.yaml to the intended model e.g.: resnet18

For simsiam, download the Sim-Siam weights based on the new model and place it under the simsiam directory. If no pre-trained simsiam weights are available, fine-tuning will take time and have to be run for more epochs. 
Change other settings as intended in config.yaml to run different configurations. Then run the application using:

In [ ]:
%env PYTHONPATH=transfer-learning/
!python anomaly_detection.py --config_file ./configs/finetuning.yaml

#### 2. Plug-in your own pre-trained customized model:

In finetuning.yaml, change 'fine_tune' flag to false and provide a custom model path under 'saved_model_path'.
Change other settings as intended in config.yaml to run different configurations.

To test the custom model with the MVTec AD dataset, add the preprocess flag to the dataset.py script to generate CSV files under all classes required for data loading:

In [ ]:
!python dataset.py -n mvtec-ad --download --preprocess -d ../../../

Then run the application using:

In [ ]:
%env PYTHONPATH=transfer-learning/
!python anomaly_detection.py --config_file ./configs/finetuning.yaml

<a id="learn-more"></a> 
## Learn More
For more information or to read about other relevant workflow examples, see these guides and software resources:
- [Intel® Transfer Learning Tool](https://github.com/IntelAI/transfer-learning)
- [Anomaly Detection fine-tuning workflow using SimSiam and CutPaste techniques](https://github.com/IntelAI/transfer-learning/tree/main/workflows/vision_anomaly_detection)
- [Intel® AI Analytics Toolkit (AI Kit)](https://www.intel.com/content/www/us/en/developer/tools/oneapi/ai-analytics-toolkit.html)
- [Intel® Extension for PyTorch](https://intel.github.io/intel-extension-for-pytorch/)
- [Intel® Extension for Scikit-learn](https://www.intel.com/content/www/us/en/developer/tools/oneapi/scikit-learn.html#gs.x609e4)
- [Intel® Neural Compressor](https://github.com/intel/neural-compressor)

<a id="support"></a> 
## Support
If you have any questions with this workflow, want help with troubleshooting, want to report a bug or submit enhancement requests, please submit a GitHub issue.

---

\*Other names and brands may be claimed as the property of others.
[Trademarks](https://www.intel.com/content/www/us/en/legal/trademarks.html).